<a href="https://colab.research.google.com/github/asantucci/Python-Workshop/blob/main/3_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install --upgrade plotly  # <-- The leading prefix indicates this is actually a shell command. This package will be imported later...

In [ ]:
# A simple read_csv example.
# (Note that this file only contains name and email...it's not intended to be a replica of what we created above...)
df = pd.read_csv("https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/names.csv")
df

# Covid-19 Analysis !

In [ ]:
# Ask pandas to read in a Comma Separated Values dataset (i.e. a spreadsheet) from the following URL,
# where we additionally specify that the first column should be treated as a Date object.
df = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv",parse_dates=[0])
print(df)

In [ ]:
# The `pivot` operation basically reshapes the data. So here, we're saying:
# Use the `date` variable as the row-index or unit-of-observation...
# ...and take each of the values in the `state` column and turn them into separate
# "features" (or columns themselves). What values should go in these columns?
# Precisely those specified by the argument which dictates that the number of cases
# should be used.
# Lastly, we replace NA's (Not Available, i.e. missing data) with Zeros.
cases_states = df.pivot(index='date',columns='state',values='cases')
cases_states = cases_states.fillna(0)
print(cases_states)  # Each row is a day, each column is a state. Values are number of cases.
# Here, we request to plot just a few columns.
cases_states.plot(y=['California','New York','Florida'])

In [ ]:
# The `diff()` function effectively takes the value in the current row and subtracts
# from it the value in the preceding row. I.e. we calculate the incremental difference
# in number of cases, by day, for each state.
daily_cases_states = cases_states.diff()
# Note that naturally the first observation(s) for each state will be missing
# (since there is no preceding value to use), and so again we fill with zeros.
daily_cases_states = daily_cases_states.fillna(0)
print(daily_cases_states)
# Here we plot the deltas.
daily_cases_states.plot(y=['California','New York','Florida'])

# Dataframes

In [ ]:
# Build a DF from a dictionary
data = {
    'Name':['Leo', 'Bob', 'John'],
    'WakeupTime':[pd.Timestamp('07:00:00'), pd.Timestamp('08:30:00'), pd.Timestamp('07:30:00')],
    'GPA':np.arange(1.0, 4.0),
    'School':'Stanford',
    'Siblings':np.array([1, 2, 0]),    
}
df = pd.DataFrame(data) 
df

## Renaming columns

In [ ]:
df = df.rename(columns = {'WakeupTime' : 'wakeup_time', 'GPA' : 'grade_point_average'})
df

## Transforming data using a function or mapping

In [ ]:
df["ineq"] = df["grade_point_average"].le(df.Siblings)  # There are plenty of builtin pandas operators!
df["cumsum"] = df.grade_point_average.cumsum()          # Before implementing a function, check for built-in methods.
df["half"] = df.grade_point_average.apply(lambda x : x / 2.0)  # You can of course also use your own functions...
print(df[["grade_point_average", "Siblings", "ineq", "cumsum", "half"]])

## Removal of duplicates in a data frame.

In [ ]:
df = pd.DataFrame({"first" : list(string.ascii_lowercase)[:5] * 2, "last" : "homogeneous"})
# Here, we've created a data.frame where all the last names are the same.
# The first names are repeated twice.
print(df)

In [ ]:
df = df.drop_duplicates()
print(df)

In [ ]:
#@title Exercise
# Extract unique rows of the following data.frame when considering columns 'a', 'b'.
df = pd.DataFrame({'a' : list(string.ascii_lowercase[:5]) * 2,
                   'b' : list(string.ascii_lowercase[:5]) * 2,
                   'c' : list(range(5*2))})
print("By construction, columns `a` and `b` are identical.\n", df)

print("If we just asked for unique rows, notice that column c has unique values...we'd return the whole DataFrame:\n",
      df.drop_duplicates())

print("But, we can also say, 'Only look at certain columns when identifying duplicates', e.g. columns 'a', 'b'\n",
      df.drop_duplicates(subset = ['a', 'b']))

## Indices

In [ ]:
# Make a column the index (and implicitly remove it as a variable, since that would be redundant).
df2 = df.set_index('Name')
df2

### Exercice

In [ ]:
# Build the dataframe where *names are the indices*
# and email & age are the columns. Adjust column names accordingly.
names  = ['Leo', 'Bob', 'Jess', 'Casey', 'John', 'Cherr']
emails = ['lc@comp.com', 'bob@stanford.edu', 'j@e.ss', 'casey@my.me', 'john@deer.us', 'cherr@y.net']
ages   = np.random.randint(1, 30, 6)

In [ ]:
#@title Solution
# Note that here, we provide columns and index separately.
df = pd.DataFrame({'Email':emails, 'Age':ages}, index=names)
df

## Join (i.e. merge)
It's very natural to want to join to data-frames together. We must have a common key in order to do so.

In [ ]:
list(string.ascii_lowercase)[:4]

In [ ]:
# As an academic example, let's examine some Census data for names in the US.
import string
males = pd.read_csv("https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/dist.male.first", delimiter=r"\s+", header = None, names = ['First', 'a', 'b', 'c'])
females = pd.read_csv("https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/dist.female.first", delimiter=r"\s+", header = None, names = ['First', 'x', 'y', 'z'])

print(males)
print(females)

In [ ]:
# We *implicitly* merge on the variable(s) that is (are) common to both tables.
males.merge(females, how = "inner")

### Types of Joins
You may be curious about different types of joins. The default argument for `merge.()` is `how = "inner"` which means to take the *intersection* between the two tables when using the specified key (which is implicitly taken to be the variable that is common to both tables).

If you specify `how = "left"`, it means "keep all of the rows that appear on the left hand side of the merge, and add the columns for the rows that match on the keys found on the right hand side of the merge. We can similarly reason about `how = "right"`.

Lastly, specifying `how = "outter"` means "keep all of the rows, i.e. the *union* of the two tables when using the specified key as argument.


## DataFrame Concatenation

In [ ]:
# Sometimes, it's handy to row-bind two data-frames together.
names = pd.concat([males, females])
print(names)

## Analyzing data

In [ ]:
# Create some data
months = pd.date_range(start='20190101', periods=12, freq='M')    # Handy utility for creating a sequence of dates. Here, we ask for 12-months of data.
change = np.random.normal(0, 1.2, (12, 3))                        # Draw from a normal distribution, mean=0 and std_dev=1.2, 12x3 matrix.
stocks = ['GOOG', 'TSLA', 'APPL']                                 # 3 different stock tickers.
df = pd.DataFrame(change, index=months, columns=stocks)
df

Quick glance at data

In [ ]:
print(df.head(3), '\n')
print(df.tail(2), '\n')
print(df.describe(), '\n')

### Indices and "Selecting" (or extracting) data

In [ ]:
## Selection using labels

# One columns
print("Extracting a column using a string to describe the column-name\n", df['GOOG'], '\n')

# A slice of rows
print("Here we request for several rows and all column alongside them...\n", df[2:5], '\n')

# Multiple rows & columns
# Endpoints INCLUDED, unlike in regular Python slicing syntax
print("Notice that the slicing works (arguably) slightly different here (endpoints _included_)...\n", 
      df.loc['2019-07-31':'2019-09-30',['TSLA','GOOG']], '\n')

In [ ]:
## Selection using conditions

print(df, '\n')
print(df.loc[df['GOOG'] > 2.5,:], '\n')         # Extracts some rows, all columns.
print(df.loc[df.index >= '2019-08-15',:], '\n') # Extracts some rows, all columns.
print(df[df > 0.5], '\n')                       # All data (i.e. perform the comparison element-wise for every element in the data-frame!)

### Groupby

In [ ]:
data = {'Name': ['Tom\'s Pizza', 'Leo\'s Taqueria', 'John\'s Burgers', 'Cindy\'s Peluqueria', 'Sergio\'s Tacos', 'Bazyli\'s Pub'],
        'Location':['NYC','SF','WDC','SF','SF','NYC'],
        'Num Customers':[5, 3, 8, 4, 6, 8],
        'Revenue':[32.6, 54.6, 43.8, 43.6, 32.6, 97.5]}
df = pd.DataFrame(data)
df

In [ ]:
groups = df.groupby('Location')
# The iteration style is as follows:
# For each "name" and "group" within the set of "groups"
for n,g in groups:
    print("-------\nGroup {}\n".format(n))
    print(g.mean())

# A similar operation could be done in one line...
# df2 = df.groupby('Location').mean()
df2 = groups.mean()
print("\nHere we print the means for each group but the results are in a data-frame...\n", df2)


## Pivot (i.e. reshape long --> wide)

In [ ]:
df = pd.DataFrame({'date'    :['2020-01-01', '2020-01-01', '2020-02-01', '2020-02-01'],
                   'crypto'  :['BTC',        'ETH',        'ETH',        'BTC'],
                   'price'   :['8192',       '350',        '405',        '9510'],
                   'exchange':['Coinbase',   'Bitconnect', 'Bitconnect', 'Bitconnect']})
df

In [ ]:
# Reshape the data...from long --> wide.
# Each observation will be a date. The columns *names* will be described by the values in the
# "crypto" column, but these variables/new-features will take on *values* as described
# by the "price" column.
df2 = df.pivot(index='date',columns='crypto',values='price')
df2

## Melt (i.e. reshape wide --> long)

In [ ]:
# You might be wondering about going the other direction...
# I.e. can we take a data.frame and go from wide --> long?
# Of course! We use the method `melt`. If we wish to use the index as the
# id-variable, we don't even need to supply any arguments to the method-call!
df2.melt()

In [ ]:
# Note that the "parse_dates" arg simply expects a list of indices describing columns
# which should be parsed as dates. The indices are zero-indexed, of course! :)
df = pd.read_csv('https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/stocks.csv',parse_dates=[1])
df

In [ ]:
# We do a little reshaping.
# Set each row to be a unique date. The columns will take on names described
# by the set of values in the "Stock" column, and the values themselves will
# be filled in using the "Open" price.
df2 = df.pivot(index='Date',columns='Stock',values='Open')
print(df2.head)
# Here, we simply plot out the opening price on each day for a couple companies...
df2.plot(y=['APPL','SBUX'])
plt.show()

## Sampling rows
This is commonly done in statistics and ML.

In [ ]:
print("Our df2 originally had ", df2.shape[0], " rows.\n")
df2.sample(frac = 1/2)  # Can also specify N explicitly. Note that no modication is done in place

### Exercise
Figure out how to sample 5 rows with replacement from `df2`.

In [ ]:
#@title Solution
# We can use arguments to the sample method.
import random                      # This import statement not really required...
random.seed(1)                     # ...we just use it to set a "seed" and reproduce random results.
df2.sample(n = 5, replace = True)  # <-- Notice that the first and last row are the same (i.e. it got resampled!)

## More on Indices

In [ ]:
# You may be wondering if it's possible to set a multi index.
# Of course this is do-able! You simply pass a list of indices.
names  = ['Leo', 'Bob', 'Jess', 'Casey', 'John', 'Cherr']
emails = ['lc@comp.com', 'bob@stanford.edu', 'j@e.ss', 'casey@my.me', 'john@deer.us', 'cherr@y.net']
ages   = np.random.randint(1, 30, 6)
df = pd.DataFrame({'Age' : ages}, index = [names, emails])
print(df)

### Exercise

In [ ]:
# Index into the first row of the data-frame using the `loc` method for a pandas
# data.frame, and don't simply use a row-number! Instead, use the index (in some way).

In [ ]:
#@title Solution
# You can use a tuple to index into a data-frame (if it has a multi-index).
df.loc[("Leo", "lc@comp.com")]

# Fancy Plotting! (More motivation than anything)

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import plotly.express as px
import json

# The "with" keyword is just syntactic sugar. We're simply opening a json file
# and initializing a variable.
with urllib.request.urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# We read in a data-frame from the web, specifying the data-types of various columns.
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', dtype={"fips": str, "state": str, "county": str})
# There is one "fips" code that needs to be corrected.
df.loc[df['county'] == "New York City",'fips'] = "36061"
# Here, we take the sum of each remaining variable in the set after using
# "fips", "state", and "county" as grouping variables.
df = df.groupby(['fips', 'state', 'county']).sum()
# Reset index is simply a way to clear any index that was previously set...
# This is useful sometimes in cleaning up after certain operations.
df = df.reset_index()
# Create a new variable that is a log (base 10) transform of the number of deaths.
df['Deaths (log10)'] = np.log10(df['deaths'])
print(df.head)

# Here's where some magic comes in...
# the counties file specified geometries / polygons (i.e. bounding boxes) 
# for each county. We use the data-frame describing deaths to colour our map.
# The details of some of these methods are beyond the scope of the course,
# but we show the example for completeness and inspiration!
fig = px.choropleth(df, locations='fips',
                        color='Deaths (log10)',
                        scope='usa',
                        geojson=counties,
                        hover_data=['deaths'])
fig.show()

# Bikes in Montreal

Link to data: https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/bikes.csv

In [ ]:
# (1)
# TODO: Properly read bikes.csv. 
# - Use ; as a separator
# - Parse the column 'Date' as dates and note that the day comes first in the CSV ("non-US" way)
# Checkout https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
# Yes it's a little scary, it would not be Pandas otherwise :-)
# Search "dayfirst" and you'll find the option
# Print the first 15 and check what you just read

In [ ]:
#@title Solution
df = pd.read_csv('https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/bikes.csv',sep=';',parse_dates=["Date"],dayfirst=True)
df = df.set_index('Date')
print(df.head(15))

In [ ]:
# (2)
# TODO: Plot the number of bikes in "du Parc" as a function of time
# Label the axes and put a title

In [ ]:
#@title Line-plot Solution
df.plot(y='du Parc')
plt.xlabel('Date')
plt.ylabel('Number of bikes')
plt.title('Du Parc\'s bikes');

In [ ]:
# (3)
# TODO (bonus): Can you try to smooth out the curve ? 
# Use a 1-week moving average.
# You'll have to google that.
# Suggestions on keywords to use: "pandas rolling average"

In [ ]:
#@title Rolling Average Line-plot Solution
df.rolling(7).mean().plot(y=['du Parc'])

In [ ]:
# (4) 
# TODO (bonus 2)
#  1. Create a column holding the weekday
#     df.index.weekday will give you that column.
#  2. Sum all cyclists in each neighborhood for each week day
#     Make a bar plot of the cyclists/day for each neighborhood
#  3. Sum all neighborhood and make a pie chart of the total number
#     of cyclists/day
# Hint: 
# - df.index.weekday returns the weekday for each date in the index
# - df.sum(axis=...) sums accross rows (axis = 0) or columns (axis = 1)
# - df.plot(kind=...) can do bar plots (kind = bar) or pie (kind = pie)

In [ ]:
df

In [ ]:
#@title Bar-plot Solution
# Clever: create a variable using the index. Since it's a date object, it has 
# a weekday index associated with it [0-6].
df['Weekday'] = df.index.weekday
# Here we calculate the sum of each column for each weekday.
df_per_week = df.groupby('Weekday').sum()
print(df_per_week)
# Bar-plot (not necessarily the most interpretable or easy to read here, but
# we show it for variety-sake...)
df_per_week.plot(kind='bar')
plt.title('Cyclists per day of the week per neighborhood')
plt.show()

In [ ]:
#@title Pie-chart Solution
# Here we calculate row-sums (where in the above Data.Frame each row was a weekday index)
# I.e. the result is the sum *across neighborhoods* for each day of the week.
df_per_week_all = df_per_week.sum(axis=1)
# Again, as a DS I would never recommend using a pie-chart (an ordered barplot
# would actually work better in this case...)
df_per_week_all.plot(kind='pie')
plt.title('Cyclists per day of the week')
plt.show()

# Extra Exercice: 311 Customer complaints

Link to data: https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/311.csv

## Read the data

In [ ]:
# TODO: Read the data, get a sense of what's in it by displaying some rows, printing columns names, etc

In [ ]:
#@title Solution
df = pd.read_csv('https://raw.githubusercontent.com/asantucci/Python-Workshop/main/data/311.csv')
print(df.head(5))
print(df.describe())
print(df.columns)

## Most common complaints

In [ ]:
# TODO: Find the 10 most common complaint and visualize the distribution of complaints
# Hint:
# - df[column].value_counts() can count the number of occurences of entries in a column

In [ ]:
#@title Solution
ct = df['Complaint Type'].value_counts()
print(ct.head(10))
ct.plot(kind='pie')

## Plumbing complaints per borough

In [ ]:
# TODO: Find the borough with the most PLUMBING complaints

In [ ]:
#@title Solution
df_p = df[df['Complaint Type'] == 'PLUMBING']
df_p_vc = df_p['Borough'].value_counts()
df_p_vc.plot(kind='bar')

## Time of complaint

In [ ]:
# (1) 'Created Date' is a string in df.
# Convert it to a proper DatetimeIndex
# and keep the hour only
# Tip: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html

# (2) Count each occurance and plot the distribution


In [ ]:
#@title Solution
hour = pd.DatetimeIndex(df['Created Date']).hour

hour_count = hour.value_counts()
hour_count.plot(kind='bar')

## Harder: Analyse complaints geographical distribution

In [ ]:
# TODO: Plot the position of the complaints
# Tip: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html

In [ ]:
#@title Solution
df.plot(kind='scatter',x='Longitude', y='Latitude')
plt.show()

In [ ]:
# (2) TODO: Plot the position of the complaints registered in Manhattan only

# (3) Round the longitude and latitude to the nearest 0.01
# Tip: np.around should be helpful

# (4) Group data by (latitude, longitude), count the size of each group
# and aggregate

# (5) Plot the (longitude, latitude, size) on a xy scatter plot


In [ ]:
#@title Solution (2)
# (2)
man = df['Borough'] == 'MANHATTAN' 
df[man].plot(kind='scatter',x='Longitude', y='Latitude')
plt.show()

In [ ]:
#@title Solution (3-4)
df.loc[man,'Longitude'] = np.around(df.loc[man,'Longitude'], 2)
df.loc[man,'Latitude'] = np.around(df.loc[man,'Latitude'], 2)
s = df.loc[man,:].groupby(['Latitude','Longitude']).size()
print(s.head(10))